In [ ]:
import gpudb
from arcgis import GIS
from arcgis import geometry
import json
import csv
import pytz
from datetime import datetime
from datetime import timedelta

In [ ]:
dte_format = '%Y-%m-%d %H:%M:%S'

In [ ]:
dte = datetime.now(tz=pytz.timezone('GMT')) - timedelta(days=1)
query_timestamp = datetime.strftime(dte,dte_format)

### setup log file

In [ ]:
# create log file
log_filename = f'{query_timestamp.replace(":", "")}.csv'
with open(log_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['datetime', 'message'])
    
def log_message(message):
    with open(log_filename, 'a+') as csv_file:
        csv_writer = csv.writer(csv_file)
        dte = datetime.now(tz=pytz.timezone('GMT')) - timedelta(days=1)
        log_ts = datetime.strftime(dte,dte_format)
        
        csv_writer.writerow([log_ts, message])

### get last run time

In [ ]:
last_run = None
with open('last-run.txt') as txt_contents:
    txt = txt_contents.read()
    last_run = datetime.strptime(txt, dte_format)

if last_run is None:
    msg = 'unable to get last run. exiting ...'
    log_message(msg)
    raise SystemExit(msg)

### Connect to Kinetica

In [ ]:
h_db = gpudb.GPUdb(
    host="admin.citizenscience.host",
    username="apfister",
    password="iswk&01Wn"
)

### Setup Kinetica query options
Use expression to only get records since the last time this script ran

In [ ]:
table_name = 'plastics_db'
limit = 1000000
encoding = 'json'
options = {
    'expression': f'captured_time > \'{last_run}\'',
    'sort_by': 'captured_time',
    'sort_order': 'DESC'
}

### Query the Plastics table

In [ ]:
log_message(f'querying kinetica :: {options["expression"]}')

recs = h_db.get_records(table_name=table_name, limit=limit, encoding=encoding, options=options)

query_record_count = recs['total_number_of_records']

log_message(f'({query_record_count}) records returned from query')

if query_record_count == 0:
    msg = 'no records returned from Kinetica'
    log_message(msg)
    raise SystemExit(msg)

### Connect to ArcGIS Online and setup feature layers

In [ ]:
ecgis = GIS(profile='ago_ec2020_py')
plastics_layer = ecgis.content.get('08878e5ab81d4074932a1069db4ded75').layers[0]
plastics_perimeter_layer = ecgis.content.get('5c952389060d4f199931f7c0622541bc').layers[0]

### Parse Inventory Data for each database record

In [ ]:
def parse_inventory_data(inv_data, cleanup_workflow):    
    inv_data_atts = {}
    try:
        inv_data_json = json.loads(inv_data)
    except:
        print ('unable to parse inventory_data')
        return inv_data_atts
    
    if cleanup_workflow == 'sample':
        for att in inv_data_json:
            for child in att['childrens']:
                field_name = f"sample_{att['id']}_{child['id']}"
                inv_data_atts[field_name] = child['value']
    elif cleanup_workflow == 'report':
        for att in inv_data_json:
            in_name = att['type'].replace(' ', '')
            in_name = in_name[0].lower() + in_name[1:]
            field_name = f'report_{in_name}'
            inv_data_atts[field_name] = att['value']
        
    return inv_data_atts    

### Create and re-project geometry of collection point

In [ ]:
def create_feature_geometry(lat, lng):
    geom = None
    try:
        geom = geometry.project([{'x': lng, 'y': lat, 'spatialReference': {'wkid': 4326}}], in_sr=4326, out_sr=3857)[0]
    except:
        print ('unable to project geometry of collection point')
        
    return geom

### Create and re-project geometry for perimeter feature

In [ ]:
def create_perimeter_feature(attributes, perimeter_json):
    feature = {
        'attributes': {},
        'geometry': None
    }
    
    # copy observation attributes into perimeter attributes
    for att in attributes:
        feature['attributes'][att] = attributes[att]
    
    # project geographic coords to web mercator
    geom = None
    try:
        geom = geometry.project([perimeter_json], in_sr=4326, out_sr=3857)[0]
        feature['geometry'] = geom
    except:
        print ('unable to project geometry of collection point')
            
    return feature

### Loop through each record from kinetica, create and stage features to add to feature services

In [ ]:
adds = []
adds_attachments = {}
perimeter_adds = []
print ('processing query results ...')
for rec in recs['records_json']:
    feature = {
        'attributes': {},
        'geometry': None
    }
    
    # load the record as a JSON object
    rec_json = json.loads(rec)
    
    # recordID
    recordID = rec_json['recordID']
    
    # individualID
    individualID = rec_json['individualID']
    
    # appInstallID
    appInstallID = rec_json['appInstallID']
    
    # captured_time
    cp_str = rec_json['captured_time']
#     cp_time = datetime.strptime(cp_str, dte_format)
    # print (cp_str, ' -- ', cp_time)
    
    # submitted_time
    sub_str = rec_json['submitted_time']
#     sub_time = datetime.strptime(sub_str, dte_format)
    # print (sub_str, ' -- ', sub_time)
    
    # image_filepath
    image_filepath = rec_json['image_filepath'] if rec_json['image_filepath'] else None

    # adding attachments from URLs won't work. only from local disk
    # we'd have to download each image locally, then reference it in the attachment upload, then delete after
    # high LOE, not much payoff. leave sample code in for now
    
#     if image_filepath is not None:
#         adds_attachments[recordID] = image_filepath
           
    # action_taken
    action_taken = rec_json['action_taken']
    
    # plastics_mode
    plastics_mode = rec_json['plastics_mode']
    
    # cleanup_workflow
    cleanup_workflow = rec_json['cleanup_workflow']
    
    # inventory_data
    inventory_data = None
    if cleanup_workflow == 'report':
        inventory_data = rec_json['inventory_data_pct']
    else:
        inventory_data = rec_json['inventory_data']
    
    inv_data_atts = parse_inventory_data(inventory_data, cleanup_workflow)
    
    # report_workflow
    report_workflow = rec_json['report_workflow']
    
    # polygon_perimeter
    polygon_perimeter = None
    try:
        polygon_perimeter = json.loads(rec_json['polygon_perimeter'])
    except:
        print (f'no perimeter found for recordID: {recordID}')
    
    # cleanup_zone_status
    cleanup_zone_status = rec_json['cleanup_zone_status']
    
    # terrain
    terrain = rec_json['terrain']
    
    # bag_count
    bag_count_json = None
    bag_count_sm = None
    bag_count_md = None
    bag_count_lg = None
    if rec_json['bag_count']:
        bag_count_json = json.loads(rec_json['bag_count'])
        bag_count_sm = int(bag_count_json['small'])
        bag_count_md = int(bag_count_json['medium'])
        bag_count_lg = int(bag_count_json['large'])
    
    # lat
    lat = None
    if rec_json['lat'] is not None:
        lat = float(rec_json['lat'])
    
    # lng
    lng = None
    if rec_json['long'] is not None:
        lng = float(rec_json['long'])
    
    # cleanup_event_id
    cleanup_event_id = int(rec_json['cleanup_event_id']) if rec_json['cleanup_event_id'] else None
    
    # cleanup_zone_id
    cleanup_zone_id = int(rec_json['cleanup_zone_id']) if rec_json['cleanup_zone_id'] else None
    
    # cleanup_grid_id
    cleanup_grid_id = int(rec_json['cleanup_grid_id']) if rec_json['cleanup_grid_id'] else None

    feature['attributes'] = {
        'recordID': recordID,
        'individualID': individualID,
        'appInstallID': appInstallID,
        'cp_time': cp_str,
        'sub_time': sub_str,
        'image_filepath': image_filepath,
        'action_taken': action_taken,
        'plastics_mode': plastics_mode,
        'cleanup_workflow': cleanup_workflow,
        'report_workflow': report_workflow,
        'cleanup_zone_status': cleanup_zone_status,
        'terrain': terrain,
        'bag_count_sm': bag_count_sm,
        'bag_count_md': bag_count_md,
        'bag_count_lg': bag_count_lg,
        'lat': lat,
        'lng': lng,
        'cleanup_event_id': cleanup_event_id,
        'cleanup_zone_id': cleanup_zone_id,
        'cleanup_grid_id': cleanup_grid_id
    }
    
    # combine inventory_data with base feature
    for att in inv_data_atts:
        feature['attributes'][att] = inv_data_atts[att]
    
    feature['geometry'] = create_feature_geometry(lat, lng)
    
    adds.append(feature)
    
    # create associated perimeter feature and stage
    if polygon_perimeter is not None:
        perimeter_feature = create_perimeter_feature(feature['attributes'], polygon_perimeter)
        perimeter_adds.append(perimeter_feature)

print ('done')

### Chunk up features into batches of 1000

In [ ]:
def chunk_it(in_list, chunk_size):
    chunked = [in_list[i * chunk_size:(i + 1) * chunk_size] for i in range((len(in_list) + chunk_size - 1) // chunk_size )] 
    return chunked

### Send updates to Plastics feature service

In [ ]:
feature_chunks = chunk_it(adds, 1000)

In [ ]:
chunk_length = len(feature_chunks)
for i, chunk in enumerate(feature_chunks):
    try:
        res = plastics_layer.edit_features(adds=chunk)
        success_msg = f'added chunk {i+1} of {chunk_length} to plastics_layer'
        log_message(success_msg)
    except Exception as e:    
        err_msg = f'error adding chunk to plastics layer :: {e}'
        log_message(err_msg)

### Send updates to Perimeter feature service

In [ ]:
perim_chunks = chunk_it(perimeter_adds, 1000)

In [ ]:
chunk_length = len(perim_chunks)
for i, chunk in enumerate(perim_chunks):
    try:
        plastics_perimeter_layer.edit_features(adds=chunk)
        success_msg = f'added chunk {i+1} of {chunk_length} to plastics_perimeter_layer'
        log_message(success_msg)
    except Exception as e:    
        err_msg = f'error adding chunk to plastics layer :: {e}'
        log_message(err_msg)

In [70]:
log_message('script completed')